In [1]:
import os  
import sys

In [2]:
sys.path.append(os.path.expanduser("~/Documents/Unity/ADS/unity-py"))

In [3]:
from unity_py.services.application_service import *

## Configuration

In [4]:
# "test" deployment of the Dockstore:
# dockstore_api_uri = "http://uads-test-dockstore-deploy-lb-1762603872.us-west-2.elb.amazonaws.com:9998/api"

# "dev" deployment of the Dockstore:
dockstore_api_uri = "http://awsLBDockstoreStack-lb-1993889266.us-west-2.elb.amazonaws.com:9998/api"

### Read Dockstore user account token from the ~/.dockstore/config file

The token for the Dockstore user account is expected to be stored in the ~/.dockstore/config file in the following format:

```
% cat ~/.dockstore/config
token 91e039fb982da7e1847fd6fc365e390de67f8eed09abc7ba606a169800eda20
```

In [5]:
# token = os.environ.get('TF_VAR_dockstore_token')

token = !cat ~/.dockstore/config  | grep token | awk '{print $2}'
token = token[0]

## Catalog Interface


In [6]:
app_catalog = DockstoreAppCatalog(dockstore_api_uri, token)

## Existing User Applications

In [7]:
for app in app_catalog.application_list(for_user=True):
    print(f"{app.id} {app.name} {app.source_repository}")

1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git
2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git


## Existing Published User Applications

In [8]:
for app in app_catalog.application_list(for_user=True, published=True):
    print(f"{app.id} {app.name} {app.source_repository}")

1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git
2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git


## Existing Published Applications for All Users

In [9]:
for app in app_catalog.application_list(published=True):
    print(f"{app.id} {app.name} {app.source_repository}")

2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git
1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git


## Create and Publish New Application in Dockstore

In [10]:
name = "workflow_test"

### Parameter Files to Upload to Dockstore

NOTE: Dockstore hard-codes the path for primary descriptor file of the hosted workflow to **Dockstore.cwl**, so we must use that filename when uploading primary CWL parameter file to the Dockstore.

**DockstoreAppCatalog** uploads all parameter files for the application at the same time (by the same URL request) when new application is registered within the Dockstore. 

In [11]:
cwl_param_files = [
    'data/dockstore_publish/Dockstore.cwl',
    'data/dockstore_publish/step.cwl'
]

json_param_files = [
    'data/dockstore_publish/parameters.json'
]

In [12]:
reg_app = app_catalog.register(app_name=name, cwl_files=cwl_param_files, json_files=json_param_files, publish=True)

In [13]:
reg_app

DockstoreApplicationPackage(name='workflow_test', source_repository='git@dockstore.org:workflows/dockstore.org/mliukis/workflow_test.git', workflow_path='/Dockstore.cwl', id='3', is_published=True, description='Bogus hosted workflow registered by unity_py.\n', workflow_type='CWL', dockstore_info={'type': 'BioWorkflow', 'descriptorType': 'CWL', 'aliases': {}, 'author': 'Masha Liukis', 'checker_id': None, 'conceptDoi': None, 'dbCreateDate': 1687902833832, 'dbUpdateDate': 1687902834396, 'defaultTestParameterFilePath': '/test.json', 'defaultVersion': '1', 'description': 'Bogus hosted workflow registered by unity_py.\n', 'descriptorTypeSubclass': 'NOT_APPLICABLE', 'email': 'mliukis@jpl.nasa.gov', 'forumUrl': None, 'full_workflow_path': 'dockstore.org/mliukis/workflow_test', 'gitUrl': 'git@dockstore.org:workflows/dockstore.org/mliukis/workflow_test.git', 'has_checker': False, 'id': 3, 'input_file_formats': [], 'isChecker': False, 'is_published': True, 'labels': [], 'lastUpdated': 16879028338

### Verify Application is Published

In [14]:
for app in app_catalog.application_list(for_user=True, published=True):
    print(f"{app.id} {app.name} {app.source_repository}")

1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git
2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git
3 workflow_test git@dockstore.org:workflows/dockstore.org/mliukis/workflow_test.git


## Upload / Update Workflow's CWL and JSON Parameter Files One at a Time 

### Upload/Update CWL File for the Application

**DockstoreAppCatalog** can upload parameter files one at a time, which might be useful when uploading updated parameter files for already registered application within the Dockstore.

In [15]:
param_file = 'data/dockstore_update/Dockstore.cwl'

In [16]:
app_catalog.upload_parameter_file(application=reg_app, param_filename=param_file)

### Upload/Update JSON File for the Application

In [17]:
json_param_file = 'data/dockstore_update/parameters.json'

In [18]:
app_catalog.upload_json_file(application=reg_app, param_filename=json_param_file)

### Remove Parameter File from the Application

To remove any of the uploaded files from the hosted workflow, please specify **None** for the local file path to upload.

In [19]:
file_to_remove = 'step.cwl'

In [20]:
app_catalog.upload_parameter_file(application=reg_app, param_filename=None, dockstore_filename=file_to_remove)

### Upload Parameter File to the Application with Provided Dockstore Filename

In [21]:
file_path = 'data/dockstore_publish/step.cwl'
dockstore_path = 'temp/test_step.cwl'

In [22]:
app_catalog.upload_parameter_file(application=reg_app, param_filename=file_path, dockstore_filename=dockstore_path)

## Unpublish Application

Dockstore does not allow to delete hosted workflows, so the application can only be unpublished.

In [23]:
app_catalog.unpublish(application=reg_app)

### Verify Application is Unpublished

In [24]:
for app in app_catalog.application_list(for_user=True, published=True):
    print(f"{app.id} {app.name} {app.source_repository}")

1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git
2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git


### Publish Application

In [25]:
app_catalog.publish(application=reg_app)

### Verify Application is Published

In [26]:
for app in app_catalog.application_list(for_user=True, published=True):
    print(f"{app.id} {app.name} {app.source_repository}")

1 workflow_example git@dockstore.org:workflows/dockstore.org/mliukis/workflow_example.git
2 workflow_for_download git@dockstore.org:workflows/dockstore.org/mliukis/workflow_for_download.git
3 workflow_test git@dockstore.org:workflows/dockstore.org/mliukis/workflow_test.git


## Upload/Update Workflow's CWL and JSON Parameter Files All at Once

In [27]:
app_catalog.upload_files(application=reg_app, cwl_files=cwl_param_files, json_files=json_param_files)

## Download Parameter Files for the Workflow

In [28]:
output_dir = f'download_application_id{reg_app.id}'

In [29]:
app_catalog.download_files(application=reg_app, output_dir_path=output_dir)

### List All Downloaded Files in "output_dir"

In [31]:
os.listdir(output_dir)

['temp',
 'parameters.json',
 'Dockstore.cwl',
 'application_id3_v6.zip',
 'step.cwl']